# Imports

In [ ]:
import torch.nn as nn
import numpy as np
import random
import torch
import torch.nn as nn
import copy
import warnings
import torch
import librosa
import csv
import random
import os
import pandas as pd
import torchaudio
import subprocess
import tempfile
from skimage.transform import resize
from skimage.filters import gaussian
from skimage.color import rgb2gray
from skimage import exposure, util
from torchvision.models import resnet101
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from sklearn.model_selection import KFold

In [ ]:
num_labels = 24
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
warnings.filterwarnings('ignore')

In [ ]:
def addChannels(img):
    return np.stack((img, img, img))

def horizontal_flip(img):
    horizontal_flip_img = img[:, ::-1]
    return addChannels(horizontal_flip_img)

def vertical_flip(img):
    vertical_flip_img = img[::-1, :]
    return addChannels(vertical_flip_img)

def addNoisy(img):
    noise_img = util.random_noise(img)
    return addChannels(noise_img)

def contrast_stretching(img):
    contrast_img = exposure.rescale_intensity(img)
    return addChannels(contrast_img)

def randomGaussian(img):
    gaussian_img = gaussian(img)
    return addChannels(gaussian_img)

def randomGamma(img):
    img_gamma = exposure.adjust_gamma(img)
    return addChannels(img_gamma)

def spec_to_image(spec):
    spec = resize(spec, (224, 400))
    eps=1e-6
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)
    spec_scaled = np.asarray(spec_scaled)
    return spec_scaled

In [ ]:
sr = 48000
length = 10 * sr
data = pd.read_csv("./dataset/train_tp.csv")

fmin = 24000
fmax = 0
for i in range(0, len(data)):
    if fmin > float(data.iloc[i]['f_min']):
        fmin = float(data.iloc[i]['f_min'])
    if fmax < float(data.iloc[i]['f_max']):
        fmax = float(data.iloc[i]['f_max'])
        
fmin = int(fmin * 0.9)
fmax = int(fmax * 1.1)

In [ ]:
def safe_load_audio(path):
    try:
        waveform, sr = torchaudio.load(path)
        return waveform.numpy().squeeze(), sr
    except Exception as e:
        print(f"[torchaudio error] {path}: {e}")
        with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmpfile:
            tmp_wav_path = tmpfile.name
        try:
            cmd = ["ffmpeg", "-y", "-i", path, tmp_wav_path]
            subprocess.run(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
            waveform, sr = torchaudio.load(tmp_wav_path)
            return waveform.numpy().squeeze(), sr
        except Exception as e2:
            print(f"[ffmpeg error] {path}: {e2}")
            return None, None
        finally:
            if os.path.exists(tmp_wav_path):
                os.remove(tmp_wav_path)

In [ ]:
label_list = []
data_list = []
audio_data = {}

for i in tqdm(range(len(data))):
    row = data.iloc[i]
    recording_id = row['recording_id']
    species_id = int(row['species_id'])
    t_min, t_max = float(row['t_min']), float(row['t_max'])

    audio_path = f"{'./dataset/train/'}{recording_id}.flac"
    wav, sr = safe_load_audio(audio_path)
    if wav is None:
        continue

    center = np.round((t_min + t_max) / 2 * sr)
    start = max(0, center - length // 2)
    end = min(len(wav), start + length)
    start = max(0, end - length)
    audio_slice = wav[int(start):int(end)]

    spec = librosa.feature.melspectrogram(y=audio_slice, sr=sr, fmin=fmin, fmax=fmax)
    spec_db = librosa.power_to_db(spec, top_db=80)
    img = spec_to_image(spec_db)

    data_list.append(recording_id)
    label_list.append(species_id)
    audio_data[recording_id] = img

In [ ]:
class AudioData(Dataset):
    def __init__(self, X, y, data_type):
        self.data = []
        self.labels = []
        self.augs = [addNoisy, contrast_stretching,randomGaussian,randomGamma, vertical_flip, horizontal_flip, addChannels]
        self.data_type=data_type
        for i in range(0, len(X)):
            recording_id = X[i]
            label = y[i]
            mel_spec = audio_data[recording_id]
            self.data.append(mel_spec)
            self.labels.append(label)
                
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.data_type == "train":
            aug= random.choice(self.augs)
            data = aug(self.data[idx])
        else:
            data = addChannels(self.data[idx])
        return data, self.labels[idx]

In [ ]:
learning_rate = 2e-4
epochs = 20
loss_fn = nn.CrossEntropyLoss()

In [ ]:
def train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, scheduler):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_losses = []
    valid_losses = []
    
    for epoch in tqdm(range(1,epochs+1)):
        model.train()
        batch_losses=[]
        for _, data in enumerate(train_loader):
            x, y = data
            optimizer.zero_grad()
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            loss.backward()
            batch_losses.append(loss.item())
            optimizer.step()
        train_losses.append(batch_losses)

        model.eval()
        batch_losses=[]
        trace_y = []
        trace_yhat = []
        
        for _, data in enumerate(valid_loader):
            x, y = data
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            trace_y.append(y.cpu().detach().numpy())
            trace_yhat.append(y_hat.cpu().detach().numpy())      
            batch_losses.append(loss.item())
        valid_losses.append(batch_losses)
        trace_y = np.concatenate(trace_y)
        trace_yhat = np.concatenate(trace_yhat)
        accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)
        
        print("epoch = %d, train_loss = %.5f, val_loss = %.5f, val_accuracy = %.5f" % (epoch, np.mean(train_losses[-1]), np.mean(valid_losses[-1]), accuracy))

        scheduler.step(np.mean(valid_losses[-1]))
        if accuracy > best_acc:
            best_acc = accuracy
            best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model

In [ ]:
nfold = 5
skf = KFold(n_splits=nfold, shuffle=True, random_state=32)

for fold_id, (train_index, val_index) in enumerate(skf.split(data_list, label_list)):
    print("Fold", fold_id)
    X_train = np.take(data_list, train_index)
    y_train = np.take(label_list, train_index, axis = 0)
    X_val = np.take(data_list, val_index)
    y_val = np.take(label_list, val_index, axis = 0)

    train_data = AudioData(X_train, y_train, "train")
    valid_data = AudioData(X_val, y_val, "valid")
    train_loader = DataLoader(train_data, batch_size=8, shuffle=True, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=8, shuffle=True, drop_last=True)

    model = resnet101(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_labels)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
    model = train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, scheduler)
    torch.save(model.state_dict(), "./model" + str(fold_id) + ".pt")
    
    del train_data, valid_data, train_loader, valid_loader, model, X_train, X_val, y_train, y_val

Fold 0


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 284MB/s]
  5%|▌         | 1/20 [00:26<08:16, 26.11s/it]

epoch = 1, train_loss = 3.00798, val_loss = 2.56881, val_accuracy = 0.25000


 10%|█         | 2/20 [00:46<06:51, 22.85s/it]

epoch = 2, train_loss = 2.71547, val_loss = 3.01856, val_accuracy = 0.25000


...

epoch = 18, train_loss = 1.21471, val_loss = 1.13709, val_accuracy = 0.78750


 95%|█████████▌| 19/20 [06:31<00:20, 20.59s/it]

epoch = 19, train_loss = 1.18850, val_loss = 1.10911, val_accuracy = 0.78750


100%|██████████| 20/20 [06:51<00:00, 20.59s/it]

epoch = 20, train_loss = 1.24220, val_loss = 1.08144, val_accuracy = 0.78750


In [ ]:
def load_test_file(f):
    wav, sr = torchaudio.load('./dataset/test/' + f, sr=None)
    wav, sr = wav.numpy(), sr
    
    segments = len(wav) / length
    segments = int(np.ceil(segments))
    
    mel_array = []
    
    for i in range(0, segments):
        if (i + 1) * length > len(wav):
            slice = wav[len(wav) - length:len(wav)]
        else:
            slice = wav[i * length:(i + 1) * length]
        
        spec=librosa.feature.melspectrogram(y=slice, sr=sr, fmin=fmin, fmax=fmax)
        spec_db=librosa.power_to_db(spec,top_db=80)

        img = spec_to_image(spec_db)
        mel_spec = np.stack((img, img, img))
        mel_array.append(mel_spec)
    
    return mel_array

In [ ]:
members = []
for i in range(nfold):
    model = resnet101(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_labels)
    model = model.to(device)
    model.load_state_dict(torch.load('./model'+str(i)+'.pt'))
    model.eval()
    members.append(model)

In [ ]:
os.remove('./model0.pt')
os.remove('./model1.pt')
os.remove('./model2.pt')
os.remove('./model3.pt')
os.remove('./model4.pt')

In [ ]:

print('Starting prediction loop')
with open('submission.csv', 'w', newline='') as csvfile:
    submission_writer = csv.writer(csvfile, delimiter=',')
    submission_writer.writerow(['recording_id','s0','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11',
                               's12','s13','s14','s15','s16','s17','s18','s19','s20','s21','s22','s23'])
    
    test_files = os.listdir('./data/test/')
    print(len(test_files))
    
    for i in range(0, len(test_files)):
        data = load_test_file(test_files[i])
        data = torch.tensor(data)
        data = data.float()
        if torch.cuda.is_available():
            data = data.cuda()

        output_list = []
        for m in members:
            output = m(data)
            maxed_output = torch.max(output, dim=0)[0]
            maxed_output = maxed_output.cpu().detach()
            output_list.append(maxed_output)
        avg_maxed_output = torch.mean(torch.stack(output_list), dim=0)
        
        file_id = str.split(test_files[i], '.')[0]
        write_array = [file_id]
        
        for out in avg_maxed_output:
            write_array.append(out.item())
    
        submission_writer.writerow(write_array)
        
        
        if i % 100 == 0 and i > 0:
            print('Predicted for ' + str(i) + ' of ' + str(len(test_files) + 1) + ' files')

print('Submission generated')

Starting prediction loop
1992
Predicted for 100 of 1993 files
Predicted for 200 of 1993 files
Predicted for 300 of 1993 files
Predicted for 400 of 1993 files
Predicted for 500 of 1993 files
Predicted for 600 of 1993 files
Predicted for 700 of 1993 files
Predicted for 800 of 1993 files
Predicted for 900 of 1993 files
Predicted for 1000 of 1993 files
Predicted for 1100 of 1993 files
Predicted for 1200 of 1993 files
Predicted for 1300 of 1993 files
Predicted for 1400 of 1993 files
Predicted for 1500 of 1993 files
Predicted for 1600 of 1993 files
Predicted for 1700 of 1993 files
Predicted for 1800 of 1993 files
Predicted for 1900 of 1993 files
Submission generated
